In [ ]:
%%capture
!pip install inductiva

# Molecular Dynamics Simulation

Until now all of our tutorials focused on fluid dynamics. However, Inductiva API purpose is to possibilitate large scale simulations simply and easily.

Therefore, here we introduce a completely different possibility with a molecular dynamics simulator: GROMACS.

Let's make the usual procedure of importing `inductiva` module and add your API key (if you still do not have your key, request one by completing the following [form](https://docs.google.com/forms/d/e/1FAIpQLSflytIIwzaBE_ZzoRloVm3uTo1OQCH6Cqhw3bhFVnC61s7Wmw/viewform?usp=sf_link)).

## Setting up Inductiva

In [ ]:
import inductiva

# Set the provided API token to be able to access our hardwarecon
inductiva.api_key = "YOUR_API_KEY"

In [ ]:
# Run this cell if you are running this notebook in Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

inductiva.working_dir = "/content/gdrive/MyDrive/demos/"

## GROMACS simulation

In this notebook, we will briefly explore how to minimize the energy of a protein configuration. This is an important in all molecular dynamics simulations and aims to find the local minimum of potential energy of the protein.

The local minimums are characterized by being structurally stable and are usually found in nature. After minimizing the energy we obtain a configuration of the protein that can be passed to molecular dynamics simulations.

### Setup simulator

In [ ]:
simulator = inductiva.molecules.GROMACS()

### Run a simulation

To minimize the energy we need the following input files:
- **sim_config_filename:** it is a `.mdp` file that contains the parameters that configure the energy minimization method (particular algorithm, time-steps, stop criteria, etc)
- **protein_filename:** `.gro` that contains the positions of the atoms of the system;
- **topology_filename:** `.top` file that contains the force-field information of the protein configuration.

With all of these we simply minimize the energy for two types of proteins: a *lysozyme* protein with the input file *1AKI_prep* and a muscular protein with the input file *3b43_ionized*.

In [ ]:
# Lysozyme protein
lysozyme_output = simulator.run(
  input_dir="config_files/gromacs",
  sim_config_filename="minim.mdp",
  protein_filename="1AKI_prep.gro",
  topology_filename="1AKI_topol.top", 
  output_dir="outputs/lysozyme_out",
)

In [ ]:
# Muscular protein
muscle_output = simulator.run(
  input_dir="config_files/gromacs",
  sim_config_filename="minim.mdp",
  protein_filename="3b43_ionized.gro",
  topology_filename="3b43_topol.top", 
  output_dir="outputs/muscle_out",
)

### Post-process outputs

The outputs of this simulation are dispersed through various files that can be used for the next molecular dynamics simulation steps.

In our particular case, we are interested in the `em.xvg` file that contains the potential energy measurements of the whole system for each time step.

We now plot the potential energy to check out the results:


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def plot_potential_energy(output_path):
  """Plot the potential energy of the system"""

  plot_path = os.path.join(output_path, "em.xvg")

  # Load data
  time_steps, potential_e = np.loadtxt(plot_path,
                                       comments=("@","#"),
                                       unpack=True)
  plt.ticklabel_format(style="scientific", scilimits=(-3,3))
  plt.xlabel("Timesteps", size = 15)
  plt.ylabel("System Potential energy (kJ/mol)", size = 15)
  plt.title("Energy minimization", size = 20)
  plt.plot(time_steps, potential_e)

In [ ]:
# Plot Lysozyme potential energy
plot_potential_energy(lysozyme_output)

In [ ]:
# Plot Muscle potential energy
plot_potential_energy(muscle_output)

## Conclusion: Easy comparison between minimal potential energies of different proteins

In this tutorial we are able to compute and compare the different minimal energies for different proteins. Moreover, we obtain an essential step for further molecular dynamics simulations that require heavy computational resources. That will be one of our next steps in the molecular dynamics world. 